## Tutorial
This file is a demonstration of the basic use of PhreeqcRM in Julia using the package `JPhreeqc.jl`. I use it for the development of extra functions that are not available in PhreeqcRM but are extremely helpful (even necessary) in working with the reactive transport model.

In [1]:
include("../src/JPhreeqc.jl")
JP = JPhreeqc

Main.JPhreeqc

## First step: create a PhreeqcRM instance
The following code shows how to create a PhreeqcRM instance that can solve chemistry in a certain number of cells. You can also specify the number of threads so the calculations run in parallel. The function RM_Create is called and the instance will get a unique id:

In [2]:
n_cells = 1
n_thread = 1
id = JP.RM_Create(n_cells, n_thread)
print("the phreeqcrm id is $id")

the phreeqcrm id is 0

The next step is to run a phreeqc input file to initialize PhreeqcRM. This can be done by running a file or a string. I prefer strings since it is easier to manipulate them and construct the required input files for, e.g. adjusting reaction equilibrium constants. The following input file is chosen from phreeqc example 8:

In [3]:
input_string = """
SURFACE_SPECIES
     Hfo_sOH  + H+ = Hfo_sOH2+
     log_k  7.18
     Hfo_sOH = Hfo_sO- + H+
     log_k  -8.82
     Hfo_sOH + Zn+2 = Hfo_sOZn+ + H+
     log_k  0.66
     Hfo_wOH  + H+ = Hfo_wOH2+
     log_k  7.18
     Hfo_wOH = Hfo_wO- + H+
     log_k  -8.82
     Hfo_wOH + Zn+2 = Hfo_wOZn+ + H+
     log_k  -2.32
SURFACE 1
     Hfo_sOH        5e-6    600.    0.09
     Hfo_wOH        2e-4
#     -Donnan
END
SOLUTION 1
     -units  mmol/kgw
     pH      8.0
     Zn      0.0001 
     Na      100.    charge 
     N(5)    100.
END
"""

"SURFACE_SPECIES\n     Hfo_sOH  + H+ = Hfo_sOH2+\n     log_k  7.18\n     Hfo_sOH = Hfo_sO- + H+\n     log_k  -8.82\n     Hfo_sOH + Zn+2 = Hfo_sOZn+ + H+\n     log_k  0.66\n     Hfo_wOH  + H+ = Hfo_wOH2+\n     log_k  7.18\n     Hfo_wOH = Hfo_wO- + H+\n     log_k  -8.82\n     Hfo_wOH + Zn+2 = Hfo_wOZn+ + H+\n     log_k  -2.32\nSURFACE 1\n     Hfo_sOH        5e-6    600.    0.09\n     Hfo_wOH        2e-4\n#     -Donnan\nEND\nSOLUTION 1\n     -units  mmol/kgw\n     pH      8.0\n     Zn      0.0001 \n     Na      100.    charge \n     N(5)    100.\nEND\n"

Now we run the above input string. But before that, we tell PhreeqcRM that it must expect surface species in the input file. Also, we set some of the PhreeqcRM default values by calling the following functions:

In [4]:
JP.setDefaultPhreeqcProperties(id)
JP.setDefaultPhreeqcUnits(id)
JP.setInitialVectors(id, n_cells, aq_solution=1, eq_phase=0, ion_exchange=0,
                             surface_site=1, gas_phase=0, solid_solution=0, kin_reaction=0)
JP.RM_LoadDatabase(id, "phreeqc.dat")


0

In [5]:
?JP.RM_RunString

Run a PHREEQC input string. The first three arguments determine which IPhreeqc instances will run the string–the workers, the InitialPhreeqc instance, and (or) the Utility instance. Input strings that modify the thermodynamic database should be run by all three sets of instances. Strings with SELECTED_OUTPUT definitions that will be used during the time-stepping loop need to be run by the workers. Strings that contain initial conditions or boundary conditions should be run by the InitialPhreeqc instance.

Parameters id	The instance id returned from RM*Create. workers	1, the workers will run the string; 0, the workers will not run the string. initial*phreeqc	1, the InitialPhreeqc instance will run the string; 0, the InitialPhreeqc will not run the string. utility	1, the Utility instance will run the string; 0, the Utility instance will not run the string. input*string	String containing PHREEQC input. Return values IRM*RESULT	0 is success, negative is failure (See RM*DecodeError). See also RM*RunFile. C Example:  strcpy(str, "DELETE; -all"); status = RM*RunString(id, 1, 0, 1, str);    // workers, initial*phreeqc, utility


In [6]:
JP.RM_RunString(id, 1,1,1, input_string)

0